In [205]:
from gtfparse import read_gtf
import pandas as pd
from collections import defaultdict, OrderedDict


In [93]:
!ls

bed_intersect_gen25-merged
Combing_GTFs
gencode.vM18.primary_assembly.annotation.gtf
gencode.vM25.primary_assembly.annotation.gff3
gencode.vM25.primary_assembly.annotation.gff3.gz
gencode.vM25.primary.gtf
GSE141750_Mse_nanopore.isoforms.gtf
GSE141750_Mse_nanopore.isoforms.gtf.gz
isoform_productivity.gtf
isoform_productivity.gz
merged_comp.gtf
Untitled.ipynb


In [44]:
chrs = ["chr" + str(num) for num in range(1,21)] + ['chrY', 'chrM', 'chrX']

In [94]:
#gencode = read_gtf("gencode.vM25.primary.gtf")
gencode = read_gtf("gencode.vM18.primary_assembly.annotation.gtf")

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'protein_id', 'ccdsid', 'ont']


In [96]:
gencode.drop(['source', 'frame', 'gene_id', 'gene_type', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag',
             'havana_transcript', 'exon_number', 'exon_id', 'protein_id', 'ccdsid', 'ont', 'level', 'havana_gene', 
              'transcript_id', 'score'], axis=1, inplace=True)

In [97]:
gencode = gencode[gencode['feature'].isin(['exon'])]
gencode = gencode[gencode['seqname'].isin(chrs)]

In [98]:
gencode

,seqname,feature,start,end,strand,gene_name
2,chr1,exon,3073253,3074322,+,RP23-271O17.1
5,chr1,exon,3102016,3102125,+,Gm26206
8,chr1,exon,3213609,3216344,-,Xkr4
9,chr1,exon,3205901,3207317,-,Xkr4
11,chr1,exon,3213439,3215632,-,Xkr4
...,...,...,...,...,...,...
1796856,chrM,exon,13552,14070,-,mt-Nd6
1796863,chrM,exon,14071,14139,-,mt-Te
1796866,chrM,exon,14145,15288,+,mt-Cytb
1796871,chrM,exon,15289,15355,+,mt-Tt


In [76]:
merged_comp = read_gtf("merged_comp.gtf")

INFO:root:Extracted GTF attributes: ['gene_id', 'transcript_id', 'exon_number', 'oId', 'tss_id']


In [77]:
merged_comp.drop(['source', 'frame', 'exon_number', 'oId', 'tss_id', 'score'], axis=1, inplace=True)

In [78]:
merged_comp

,seqname,feature,start,end,strand,gene_id,transcript_id
0,chr1,exon,3073253,3074322,+,XLOC_000001,TCONS_00000001
1,chr1,exon,3102016,3102125,+,XLOC_000002,TCONS_00000002
2,chr1,exon,3680155,3681788,+,XLOC_000003,TCONS_00000003
3,chr1,exon,3752010,3754360,+,XLOC_000004,TCONS_00000004
4,chr1,exon,4496551,4499378,+,XLOC_000005,TCONS_00000005
...,...,...,...,...,...,...,...
999707,chrY,exon,90665346,90665414,-,XLOC_048379,TCONS_00135677
999708,chrY,exon,90666856,90667037,-,XLOC_048379,TCONS_00135677
999709,chrY,exon,90667525,90667625,-,XLOC_048379,TCONS_00135677
999710,chrY,exon,90754513,90754821,-,XLOC_048380,TCONS_00135679


In [257]:
merged_comp.to_csv("XLOC.tsv",index=False, sep="\t")

In [240]:
merged = pd.merge(gencode, merged_comp,  how='outer', left_on=['start','end'], right_on = ['start','end'])

In [241]:
merged

,seqname_x,feature_x,start,end,strand_x,gene_name,seqname_y,feature_y,strand_y,gene_id,transcript_id
0,chr1,exon,3073253,3074322,+,RP23-271O17.1,chr1,exon,+,XLOC_000001,TCONS_00000001
1,chr1,exon,3102016,3102125,+,Gm26206,chr1,exon,+,XLOC_000002,TCONS_00000002
2,chr1,exon,3213609,3216344,-,Xkr4,NaN,NaN,NaN,NaN,NaN
3,chr1,exon,3205901,3207317,-,Xkr4,chr1,exon,-,XLOC_001549,TCONS_00004177
4,chr1,exon,3205901,3207317,-,Xkr4,chr1,exon,-,XLOC_001549,TCONS_00004176
...,...,...,...,...,...,...,...,...,...,...,...
2855966,NaN,NaN,73346504,73347038,NaN,NaN,chrY,exon,-,XLOC_048158,TCONS_00135437
2855967,NaN,NaN,73346504,73347038,NaN,NaN,chrY,exon,-,XLOC_048158,TCONS_00135443
2855968,NaN,NaN,74041330,74041861,NaN,NaN,chrY,exon,-,XLOC_048166,TCONS_00135455
2855969,NaN,NaN,74041330,74041861,NaN,NaN,chrY,exon,-,XLOC_048166,TCONS_00135454


In [256]:
merged[merged['gene_id'] == "XLOC_001812"]


,seqname_x,feature_x,start,end,strand_x,gene_name,seqname_y,feature_y,strand_y,gene_id,transcript_id
2590794,NaN,NaN,37485851,37488982,NaN,NaN,chr1,exon,-,XLOC_001812,TCONS_00004789


In [243]:
merged[merged['gene_name'] == "RP23-6C3.2"]


,seqname_x,feature_x,start,end,strand_x,gene_name,seqname_y,feature_y,strand_y,gene_id,transcript_id
2513716,chrX,exon,60706270,60707063,-,RP23-6C3.2,chrX,exon,nan,XLOC_046825,TCONS_00133540


In [245]:
#merged[~merged['gene_name'].notna()]
#merged[merged['transcript_id'] == 'TCONS_00000011']
reduced = merged.drop(['seqname_x', 'feature_x', 'start', 'end', 'strand_x', 'strand_y', 'seqname_y', 'feature_y', 'transcript_id'], axis=1)

In [246]:
#len(reduced.drop_duplicates()['gene_name'].tolist())
len(set(reduced.drop_duplicates()['gene_name'].tolist()))

54088

In [247]:
reduced.drop_duplicates(inplace=True)

In [248]:
xlocs = reduced.drop_duplicates()['gene_id'].tolist()
xlocs = sorted([i for i in xlocs if not pd.isna(i)])

In [251]:
translation_dict = defaultdict(list)

for i in xlocs:
    id_list = (reduced[reduced['gene_id'] == i]['gene_name'].tolist())#(index=False))
    for l in id_list:
        if l not in translation_dict[i] and not pd.isna(l):
            translation_dict[i].append(l)

In [252]:
sort_t_dict = OrderedDict(sorted(translation_dict.items()))

In [253]:
len(set(xlocs)) == len(sort_t_dict)

True

In [254]:
i = 0
with open("XLOC_to_Gencode_18.txt", 'w') as f:
    for k,v in sort_t_dict.items():
        f.write(k + "\t" + ",".join(v) + '\n')

In [65]:
def count_matched(tsv_in):
    with open (tsv_in, 'r') as f:
        matched_count, missing_count = 0, 0
        for s in f.readlines():
            s = s.strip().split("\t")
            if len(s) == 1: missing_count+=1
            else: matched_count+=1
        print("Missing:\t{}\nMatched:\t{}\nPercent Matched\t{:.2f}%".format(missing_count, matched_count, (matched_count - missing_count)/matched_count * 100))

In [66]:
count_matched('XLOC_to_Gencode_18.txt')

Missing:	1245
Matched:	47136
Percent Matched	97.36%


In [37]:
convert_to_sed('XLOC_to_Gencode_18.txt', 'XLOC_to_Gencode_18_SED')

In [48]:
def convert_to_sed(tsv_in, sed_out):
    str_out="s/{}/{}/g\n"
    with open (sed_out, 'w+') as o:
        with open (tsv_in, 'r') as f:
            entries = [i.strip().split("\t") for i in f.readlines()]
            o.writelines([str_out.format(e[0], e[0]) if len(e) == 1 else str_out.format(e[0], e[1]) for e in entries])

In [49]:
convert_to_sed('XLOC_to_Gencode_18.txt', 'XLOC_to_Gencode_18_SED')